In [3]:
!pip install Langchain
!pip install langchain-huggingface
!pip install huggingface_hub

In [4]:
from google.colab import userdata
import os
os.environ['HUGGINGFACEHUB_API_TOKEN'] = userdata.get('HUGGINGFACEHUB_API_TOKEN')

# Sequential_chain

In [8]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', temperature=0.4)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt = PromptTemplate(
    template='''
    Generate a short poem about {topic}.
    ''',
    input_variables={'topic'}

)

prompt_1 = PromptTemplate(
    template='''
    Summarize the following poem:
    {poem}
    ''',
    input_variables={'poem'}
)

chain = prompt | model | parser | prompt_1 | model | parser
chain.invoke({'topic':'Football'})

" This poem captures the excitement and passion of a football game. It describes the joy of the crowd, the beauty of the game in motion, and the intense emotions experienced by the players. The poem highlights the camaraderie and strength displayed on the field, and the way in which football brings dreams to life. The premise of the game is a source of courage, spirit, and unyielding play, with the victor's triumph illuminating the night."

# Parallal Chain
run multipale parallal

In [11]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain.schema.runnable import RunnableParallel,RunnablePassthrough

llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', temperature=0.4)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

prompt = PromptTemplate(
    template='''
    Generate a short poem about {topic}.
    ''',
    input_variables={'topic'}

)

prompt_1 = PromptTemplate(
    template='''
    Summarize the following poem:
    {poem}
    ''',
    input_variables={'poem'}
)

chain = prompt | model | parser
Parallal_chain = RunnableParallel({
    "poem":RunnablePassthrough(),  # a Runnable that takes an input and returns the same input unchanged is achieved using RunnablePassthrough
    "summary":prompt_1 | model | parser
})

marge_chain = chain | Parallal_chain

result = marge_chain.invoke({'topic':'Football'})

print(result['poem'])
print(f"summaery of the poem : {result['summary']}")

 In the heart of autumn, under the sun's golden gleam,
Stands a field of passion, filled with triumphant dream.
Football's call echoes, through the crisp autumn air,
Bringing together hearts and spirits, beyond compare.

A dance of strength and skill, in the vibrant arena,
Eleven faces focused, hearts flying free like eagle's leap.
The crowd's roar shakes the earth, in anticipation's throe,
A symphony of passion that only the brave dare to know.

A tale of strategy, of unyielding will,
An art form brought to life, with every desperate, thrilling spill.
Through sweat and triumph, and heartache, hand in hand,
We cheer for our heroes, in our beautiful, timeless land.

In the heart of the field, the magic unfolds,
Every play, every pass, every step bravely told.
Football, a field of dreams, a dance of fire and ice,
Football, a symphony of spirit, igniting our inner child with grace.
summaery of the poem :  This poem celebrates the excitement and passion of football during autumn. The poem 

# Conditional Chain
chain will run wnen condition are satisfied

In [24]:
from langchain_huggingface import HuggingFaceEndpoint,ChatHuggingFace
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import PydanticOutputParser
from langchain.schema.runnable import RunnableBranch,RunnablePassthrough
from pydantic import BaseModel,Field
from typing import Literal

llm = HuggingFaceEndpoint(repo_id='mistralai/Mistral-7B-Instruct-v0.2', temperature=0.4)
model = ChatHuggingFace(llm=llm)

parser = StrOutputParser()

class sentimantes(BaseModel):
  sentiment: Literal['positive', 'negative'] = Field(description='sentiment of the review')

parser_1  = PydanticOutputParser(pydantic_object=sentimantes)

prompt = PromptTemplate(
    template='''
    Give me the sentiment of the following review ':
    {review} \n {format_instruction}
    ''',
    input_variables={'review'},
    partial_variables={'format_instruction':parser_1.get_format_instructions()}

)

prompt_1 = PromptTemplate(
    template='''
    write a appropiate positve feedback for the following review:
    {review}
    ''',
    input_variables={'review'}
)

prompt_2 = PromptTemplate(
    template='''
    write a appropiate negative feedback for the following review:
    {review}
    ''',
    input_variables={'review'}
)

chain = prompt | model | parser_1

chain_1 = prompt_1 | model | parser
chain_2 = prompt_2 | model | parser

Condition_chain = RunnableBranch(
    (lambda X: X.sentiment == 'positive', chain_1),
    (lambda X: X.sentiment== 'negative', chain_2),
    RunnablePassthrough()
)

marge_chain = chain | Condition_chain

result = marge_chain.invoke({'review':'This is most underrated  phone '})

print(result)

 I'm glad to hear that you had a positive experience. Your feedback is greatly appreciated as it helps us continue to provide excellent service. Keep sharing your wonderful experiences and we look forward to serving you again soon!
